In [ ]:
#Task-4 solution

from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *
import math
import random


scrn_w, scrn_h = 1000, 800


# camera stuff
cam_xyz = [0, 300, 300]
fov_deg = 120
MAP_SIZE = 600
rnd_num = 423
# player
plyr_x = 0.0
plyr_z = 0.0
plyr_rot = 0.0
cam_rot = 0.0
cam_hght = 300.0
fps_mode = False
# gme mechanics
hp = 5
pts = 0
missd_sht = 0
gme_end = False
hax_on = False
auto_trgt = False
# projctl list
shots = []
# bad_guys list
bad_guys = []
nme_size_time = 0.0
# anime vars
hax_spin = 0.0
anime_cnt = 0

def make_enemy():
    """nme banano"""
    global bad_guys
    bad_guys = [BadGuy() for _ in range(5)]
    print(" BATTLE INITIALIZED SUCCESSFULLY!")
    print(" 5 hostile targets spawned on battlefield!")

def get_x_pct(prcnt):
    """pct to pixel"""
    return int(scrn_w * prcnt / 100.0)

def get_y_pct(prcnt):
    """pct to pixel"""
    return int(scrn_h * prcnt / 100.0)

def draw_txt_at(x_p, y_p, txt, z=0.1, fnt=GLUT_BITMAP_HELVETICA_18):
    """txt draw kora"""
    x = get_x_pct(x_p)
    y = get_y_pct(y_p)

    glColor3f(1, 1, 1)
    glMatrixMode(GL_PROJECTION)
    glPushMatrix()
    glLoadIdentity()
    gluOrtho2D(0, scrn_w, 0, scrn_h)

    glMatrixMode(GL_MODELVIEW)
    glPushMatrix()
    glLoadIdentity()

    glTranslatef(0, 0, z)
    glRasterPos2f(x, y)
    for ch in txt:
        glutBitmapCharacter(fnt, ord(ch))

    glPopMatrix()
    glMatrixMode(GL_PROJECTION)
    glPopMatrix()
    glMatrixMode(GL_MODELVIEW)

class Projctl:            # bullet class
    def __init__(self, x, z, angl):
        self.x = x
        self.z = z
        self.angl = angl
        self.spd = 8.0
        self.alive = True

    def updt(self):
        """goli move kora"""
        if self.alive:
            # goli chole
            self.x += self.spd * math.sin(math.radians(self.angl))
            self.z += self.spd * math.cos(math.radians(self.angl))

            # boundary chk
            if abs(self.x) > MAP_SIZE or abs(self.z) > MAP_SIZE:
                self.alive = False
                return False
        return True

class BadGuy:           # enemy class
    def __init__(self):
        self.reset_pos()
        self.spd = 0.1

    def updt(self, trgt_x, trgt_z):
        """plyr dike ase"""
        dx = trgt_x - self.x
        dz = trgt_z - self.z
        dist = math.sqrt(dx*dx + dz*dz)

        if dist > 5.0:
            self.x += (dx / dist) * self.spd
            self.z += (dz / dist) * self.spd

    def reset_pos(self):
        """notun pos"""
        self.x = random.uniform(-MAP_SIZE + 50, MAP_SIZE - 50)
        self.z = random.uniform(-MAP_SIZE + 50, MAP_SIZE - 50)
        print(" HOSTILE TARGET relocated to new coordinates!")

def draw_hud_info():
    """status dekhano"""
    # jaan txt
    glColor3f(1.0, 0.0, 0.0)
    draw_txt_at(2, 95, f"HEALTH: {hp}", fnt=GLUT_BITMAP_HELVETICA_18)

    # score txt
    glColor3f(1.0, 1.0, 0.0)
    draw_txt_at(2, 90, f"POINTS: {pts}", fnt=GLUT_BITMAP_HELVETICA_18)

    # miss count
    glColor3f(1.0, 0.5, 0.5)
    draw_txt_at(2, 85, f"Missed: {missd_sht}/10")

def draw_ctrl_info():
    """control dekhano"""
    # control txt
    glColor3f(0.9, 0.8, 1.0)  # lite purple
    draw_txt_at(83, 94, "WASD: Move/Rotate", fnt=GLUT_BITMAP_HELVETICA_10)
    draw_txt_at(83, 91, "Arrows: Camera View", fnt=GLUT_BITMAP_HELVETICA_10)
    draw_txt_at(83, 88, "Mouse: Shoot/Toggle", fnt=GLUT_BITMAP_HELVETICA_10)
    draw_txt_at(83, 85, "C: Hack  R: Reset", fnt=GLUT_BITMAP_HELVETICA_10)

def draw_gme_end_box():

    lft = get_x_pct(35)
    rght = get_x_pct(60)
    btm = get_y_pct(45)
    tp = get_y_pct(55)

    glMatrixMode(GL_PROJECTION)
    glPushMatrix()
    glLoadIdentity()
    gluOrtho2D(0, scrn_w, 0, scrn_h)

    glMatrixMode(GL_MODELVIEW)
    glPushMatrix()
    glLoadIdentity()

    # bg
    glColor3f(0.0, 0.0, 0.0)
    glBegin(GL_QUADS)
    glVertex2f(lft, btm)
    glVertex2f(rght, btm)
    glVertex2f(rght, tp)
    glVertex2f(lft, tp)
    glEnd()



    glPopMatrix()
    glMatrixMode(GL_PROJECTION)
    glPopMatrix()
    glMatrixMode(GL_MODELVIEW)

def render_plyr():
    """plyr draw"""
    global gme_end, plyr_x, plyr_z, plyr_rot, fps_mode

    glPushMatrix()
    glTranslatef(plyr_x, plyr_z, 0)

    if gme_end:
        glRotatef(90, 1, 0, 0)  # shuye pore

    curr_angl = hax_spin if hax_on else plyr_rot
    glRotatef(-curr_angl, 0, 0, 1)
        # bam juta
    glColor3f(0.2, 0.2, 0.2)
    glPushMatrix()
    glTranslatef(-8, 0, 1)
    glScalef(1.2, 1.8, 0.6)
    glutSolidCube(6)
    glPopMatrix()

    # dan juta
    glColor3f(0.2, 0.2, 0.2)
    glPushMatrix()
    glTranslatef(8, 0, 1)
    glScalef(1.2, 1.8, 0.6)
    glutSolidCube(6)
    glPopMatrix()
        # bam pa
    glColor3f(0.2, 0.2, 0.9)
    glPushMatrix()
    glTranslatef(-8, 0, 10)
    gluCylinder(gluNewQuadric(), 4, 4, 30, 8, 8)
    glPopMatrix()

    # dan pa
    glColor3f(0.2, 0.2, 0.9)
    glPushMatrix()
    glTranslatef(8, 0, 10)
    gluCylinder(gluNewQuadric(), 4, 4, 30, 8, 8)
    glPopMatrix()


    if fps_mode == False:


        # gaye - light green
        glColor3f(0.2, 0.8, 0.3)
        glPushMatrix()
        glTranslatef(0, 0, 45)
        glScalef(1.0, 0.8, 1.2)
        glutSolidCube(20)
        glPopMatrix()

        # bam hat
        glColor3f(0.9, 0.7, 0.6)
        glPushMatrix()
        glTranslatef(-15, 0, 50)
        glRotatef(-90, 1, 0, 0)
        gluCylinder(gluNewQuadric(), 3, 3, 25, 8, 8)
        glPopMatrix()

        # dan hat
        glColor3f(0.9, 0.7, 0.6)
        glPushMatrix()
        glTranslatef(15, 0, 50)
        glRotatef(-90, 1, 0, 0)
        gluCylinder(gluNewQuadric(), 3, 3, 25, 8, 8)
        glPopMatrix()
                # matha
        glColor3f(0.9, 0.7, 0.6)
        glPushMatrix()
        glTranslatef(0, 0, 70)
        glutSolidSphere(12, 10, 10)
        glPopMatrix()
        # gun nal - lite silver
        glColor3f(0.9, 0.9, 0.9)  # lite
        quadrc = gluNewQuadric()
        glPushMatrix()
        glTranslatef(0, 18, 50)
        glRotatef(-90, 1, 0, 0)
        gluCylinder(quadrc, 2.0, 1.5, 25, 12, 6)
        glPopMatrix()
        gluDeleteQuadric(quadrc)

        # gun details - lite gray
        glColor3f(0.8, 0.8, 0.8)  # lite
        for ring_pos in [6, 12, 18]:
            glPushMatrix()
            glTranslatef(0, 18+ ring_pos, 50)
            glRotatef(-90, 1, 0, 0)
            quadrc = gluNewQuadric()
            gluCylinder(quadrc, 2.1, 2.1, 1, 8, 2)
            gluDeleteQuadric(quadrc)
            glPopMatrix()

        # gun tip
        glColor3f(0.9, 0.9, 0.9)
        glPushMatrix()
        glTranslatef(0, 18, 50)
        glutSolidSphere(2, 12, 12)
        glPopMatrix()

        # sight
        glColor3f(0.6, 0.6, 0.6)
        glPushMatrix()
        glTranslatef(0, 16, 52)
        glScalef(0.3, 0.8, 0.2)
        glutSolidCube(4)
        glPopMatrix()

    else:
        # fps gun - short
        glColor3f(0.9, 0.9, 0.9)  # lite
        quadrc = gluNewQuadric()
        glPushMatrix()
        glTranslatef(0, 18, 50)
        glRotatef(-90, 1, 0, 0)
        gluCylinder(quadrc, 2.0, 1.5, 15, 12, 6)
        glPopMatrix()
        gluDeleteQuadric(quadrc)

        # gun tip
        glColor3f(0.7, 0.7, 0.7)  # lite
        glPushMatrix()
        glTranslatef(0, 18+14, 55)
        glutSolidSphere(0.5, 10, 10)
        glPopMatrix()

    # gun handle
    glColor3f(0.6, 0.6, 0.6)
    glPushMatrix()
    glTranslatef(0, 18, 47)
    glScalef(0.6, 0.4, 0.8)
    glutSolidCube(6)
    glPopMatrix()

    # trigger
    glColor3f(0.5, 0.5, 0.5)
    glPushMatrix()
    glTranslatef(0, 15, 44)
    glScalef(0.3, 0.2, 0.5)
    glutSolidCube(4)
    glPopMatrix()

    # grip texture
    glColor3f(0.6, 0.6, 0.6)
    for i, grp_z in enumerate([44, 46, 48, 50]):
        glPushMatrix()
        glTranslatef(1.5, 18, grp_z)
        glScalef(0.08, 0.2, 0.15)
        glutSolidCube(2)
        glPopMatrix()

        glPushMatrix()
        glTranslatef(-1.5, 18, grp_z)
        glScalef(0.08, 0.2, 0.15)
        glutSolidCube(2)
        glPopMatrix()



    glPopMatrix()

def render_baddies():
    """enemy draw"""
    global nme_size_time, anime_cnt
    anime_cnt += 1
    nme_size_time = anime_cnt * 0.02

    # sort by distance
    sorted_nmes = sorted(bad_guys, key=lambda nme: math.sqrt((plyr_x - nme.x)**2 + (plyr_z - nme.z)**2), reverse=True)

    for i, nme in enumerate(sorted_nmes):
        glPushMatrix()
        glTranslatef(nme.x, nme.z, 20)

        # body - lite red
        scl_fact = 1.0 + 0.1 * math.cos(nme_size_time + i * 2)
        glColor3f(1.0, 0.4, 0.4)
        glPushMatrix()
        glScalef(scl_fact, scl_fact, scl_fact)
        glutSolidSphere(20, 10, 10)
        glPopMatrix()

        # head - lite gray
        glColor3f(0.3, 0.3, 0.3)
        glPushMatrix()
        glTranslatef(0, 0, 25)
        glScalef(scl_fact * 0.7, scl_fact * 0.7, scl_fact * 0.7)
        glutSolidSphere(12, 8, 8)
        glPopMatrix()

        glPopMatrix()

def render_shots():
    """goli draw"""
    for sht in shots:
        if sht.alive:
            glPushMatrix()
            glTranslatef(sht.x, sht.z, 11)

            glRotatef(-sht.angl + 90, 0, 0, 1)

            # plasma core - lite blue
            glColor3f(0.4, 0.9, 1.0)
            glPushMatrix()
            glTranslatef(0, 0, 2.5)
            glScalef(0.6, 0.6, 2.0)
            glutSolidSphere(1.2, 8, 8)
            glPopMatrix()

            # energy - lite cyan
            glColor3f(0.3, 0.8, 0.9)
            glPushMatrix()
            glTranslatef(0, 0, 0.5)
            gluCylinder(gluNewQuadric(), 1.0, 0.6, 3, 8, 8)
            glPopMatrix()

            # base - lite gray
            glColor3f(0.4, 0.4, 0.5)
            glPushMatrix()
            glTranslatef(0, 0, -1.0)
            glScalef(1.2, 1.2, 1.0)
            glutSolidCube(4)
            glPopMatrix()

            glPopMatrix()

def render_floor_n_walls():
    """floor ar wall"""
    # chess board floor
    tile_sz = 50
    num_tiles = int((MAP_SIZE * 2) / tile_sz)

    for i in range(num_tiles):
        for j in range(num_tiles):
            x_st = -MAP_SIZE + i * tile_sz
            z_st = -MAP_SIZE + j * tile_sz

            if (i + j) % 2 == 0:
                glColor3f(1.0, 1.0, 1.0)
            else:
                glColor3f(0.8, 0.7, 1.0)

            glBegin(GL_QUADS)
            glVertex3f(x_st, z_st, 0)
            glVertex3f(x_st + tile_sz, z_st, 0)
            glVertex3f(x_st + tile_sz, z_st + tile_sz, 0)
            glVertex3f(x_st, z_st + tile_sz, 0)
            glEnd()

    # walls
    wall_hght = 80

    # front
    glColor3f(0.76, 0.14, 0.61)  # pink
    glBegin(GL_QUADS)
    glVertex3f(-MAP_SIZE, MAP_SIZE, 0)
    glVertex3f(MAP_SIZE, MAP_SIZE, 0)
    glVertex3f(MAP_SIZE, MAP_SIZE, wall_hght)
    glVertex3f(-MAP_SIZE, MAP_SIZE, wall_hght)
    glEnd()

    # back
    glColor3f(0.06, 0.78, 0.73)  #cyan
    glBegin(GL_QUADS)
    glVertex3f(-MAP_SIZE, -MAP_SIZE, 0)
    glVertex3f(-MAP_SIZE, -MAP_SIZE, wall_hght)
    glVertex3f(MAP_SIZE, -MAP_SIZE, wall_hght)
    glVertex3f(MAP_SIZE, -MAP_SIZE, 0)
    glEnd()

    # left
    glColor3f(0.15, 0.15, 0.82)  # blue
    glBegin(GL_QUADS)
    glVertex3f(-MAP_SIZE, -MAP_SIZE, 0)
    glVertex3f(-MAP_SIZE, MAP_SIZE, 0)
    glVertex3f(-MAP_SIZE, MAP_SIZE, wall_hght)
    glVertex3f(-MAP_SIZE, -MAP_SIZE, wall_hght)
    glEnd()

    # right
    glColor3f(0.8, 0.45, 0.11)  # orange
    glBegin(GL_QUADS)
    glVertex3f(MAP_SIZE, -MAP_SIZE, 0)
    glVertex3f(MAP_SIZE, -MAP_SIZE, wall_hght)
    glVertex3f(MAP_SIZE, MAP_SIZE, wall_hght)
    glVertex3f(MAP_SIZE, MAP_SIZE, 0)
    glEnd()

def chk_hit():
    """collision check"""
    global shots, bad_guys, pts, hp, gme_end

    # bullet hit enemy
    for sht in shots[:]:
        if sht.alive:
            for nme in bad_guys:
                dist = math.sqrt((sht.x - nme.x)**2 + (sht.z - nme.z)**2)
                if dist < 25:
                    sht.alive = False
                    shots.remove(sht)
                    nme.reset_pos()
                    pts += 10
                    mssg = f" PERFECT SHOT! Target eliminated! Score: {pts}"
                    print(mssg)
                    break

    # plyr hit enemy
    for nme in bad_guys:
        dist = math.sqrt((plyr_x - nme.x)**2 + (plyr_z - nme.z)**2)
        if dist < 40:
            hp -= 1
            nme.reset_pos()
            mssg = f" DAMAGE TAKEN! Enemy contact! Health remaining: {hp}"
            print(mssg)
            if hp <= 0:
                gme_end = True
                mssg = " MISSION FAILED! All health depleted!"
                print(mssg)

    # remove dead shots
    for sht in shots[:]:
        if not sht.alive:
            shots.remove(sht)

def shoot_goli():
    """goli fire"""
    global shots, hax_on, plyr_rot, hax_spin

    if hax_on:
        angl = hax_spin
        mssg = " PLASMA ROUND FIRED! Auto-targeting system!"
        print(mssg)
    else:
        angl = plyr_rot
        mssg = " PLASMA ROUND FIRED! Manual targeting!"
        print(mssg)

    goli = Projctl(plyr_x, plyr_z, angl)
    shots.append(goli)

def chk_nme_sight():
    """enemy sight check"""
    curr_angl = hax_spin if hax_on else plyr_rot

    for nme in bad_guys:
        dx = nme.x - plyr_x
        dz = nme.z - plyr_z
        dist = math.sqrt(dx*dx + dz*dz)

        if dist < 15.0 or dist > 600.0:
             continue

        nme_angl = math.degrees(math.atan2(dx, dz))

        plyr_norm = curr_angl % 360
        nme_norm = nme_angl % 360

        angl_diff = abs(plyr_norm - nme_norm)
        if angl_diff > 180:
            angl_diff = 360 - angl_diff

        if angl_diff < 1:
            return True, nme

    return False, None

def updt_gme():
    """game update"""
    global hax_spin, shots, anime_cnt, missd_sht, gme_end

    if not gme_end:

        for nme in bad_guys:
            nme.updt(plyr_x, plyr_z)

        for sht in shots[:]:
            if not sht.updt():
                shots.remove(sht)
                missd_sht += 1
                mssg = " SHOT MISSED! Projectile out of bounds"
                print(mssg)
                if missd_sht >= 10:
                    gme_end = True
                    mssg = " EXCESSIVE MISSES! Mission terminated!"
                    print(mssg)

        if hax_on:
            hax_spin += 2.0
            if hax_spin >= 360:
                hax_spin = 0

            nme_sight, trgt_nme = chk_nme_sight()

            if nme_sight and trgt_nme and len(shots) < 15:
                shoot_goli()
                mssg = f" PRECISION SHOT: Target acquired and locked!"
                print(mssg)

        chk_hit()

def kybrd_input(key, x, y):
    """keyboard input"""
    global plyr_x, plyr_z, plyr_rot, hax_on, auto_trgt
    global hp, pts, missd_sht, gme_end

    if isinstance(key, bytes):
        key = key.lower()

    if not gme_end:
        #(W key)
        if key == b'w':
            old_x, old_z = plyr_x, plyr_z
            plyr_x += 8.0 * math.sin(math.radians(plyr_rot))
            plyr_z += 8.0 * math.cos(math.radians(plyr_rot))
            plyr_x = max(-MAP_SIZE + 30, min(MAP_SIZE - 30, plyr_x))
            plyr_z = max(-MAP_SIZE + 30, min(MAP_SIZE - 30, plyr_z))
            if old_x != plyr_x or old_z != plyr_z:
                mssg = " ADVANCING FORWARD!"
                print(mssg)

        #(S key)
        if key == b's':
            old_x, old_z = plyr_x, plyr_z
            plyr_x -= 8.0 * math.sin(math.radians(plyr_rot))
            plyr_z -= 8.0 * math.cos(math.radians(plyr_rot))
            plyr_x = max(-MAP_SIZE + 30, min(MAP_SIZE - 30, plyr_x))
            plyr_z = max(-MAP_SIZE + 30, min(MAP_SIZE - 30, plyr_z))
            if old_x != plyr_x or old_z != plyr_z:
                mssg = " RETREATING BACKWARD!"
                print(mssg)

        #(A key)
        if key == b'a':
            if not hax_on:
                plyr_rot -= 4.0
                mssg = "↺ ROTATING LEFT!"
                print(mssg)

        #(D key)
        if key == b'd':
            if not hax_on:
                plyr_rot += 4.0
                mssg = "↻ ROTATING RIGHT!"
                print(mssg)

        #(C key)
        if key == b'c':
            hax_on = not hax_on
            if hax_on:
                mssg = " HACK MODE ACTIVATED! Auto-aim system online!"
                print(mssg)
            else:
                mssg = " HACK MODE DEACTIVATED! Manual control restored!"
                print(mssg)

        # (V key)camera move
        if key == b'v':
            if hax_on:
                auto_trgt = not auto_trgt
                if auto_trgt:
                    mssg = " AUTO-TRACKING VISION ENABLED!"
                    print(mssg)
                else:
                    mssg = " AUTO-TRACKING VISION DISABLED!"
                    print(mssg)

    #(R key)
    if key == b'r':
        if gme_end:
            hp = 5
            pts = 0
            missd_sht = 0
            gme_end = False
            hax_on = False
            auto_trgt = False
            plyr_x = 0.0
            plyr_z = 0.0
            plyr_rot = 0.0
            shots.clear()
            for nme in bad_guys:
                nme.reset_pos()
            mssg = " MISSION RESET! New operation initiated!"
            print(mssg)

def spcl_key_input(key, x, y):
    """special key input"""
    global cam_xyz, plyr_rot, fps_mode

    if fps_mode:
        if key == GLUT_KEY_LEFT:
            plyr_rot -= 5.0
            mssg = " SCANNING LEFT!"
            print(mssg)

        if key == GLUT_KEY_RIGHT:
            plyr_rot += 5.0
            mssg = " SCANNING RIGHT!"
            print(mssg)

        if key == GLUT_KEY_UP:
            mssg = " LOOKING UP!"
            print(mssg)

        if key == GLUT_KEY_DOWN:
            mssg = " LOOKING DOWN!"
            print(mssg)
    else:
        # 3rd person mode
        x, y, z = cam_xyz

        rad = math.sqrt(x**2 + y**2)

        if rad < 50:
            rad = 300
            x = rad
            y = 0

        if key == GLUT_KEY_UP:
            z += 20
            mssg = " CAMERA ELEVATION UP!"
            print(mssg)

        if key == GLUT_KEY_DOWN:
            z -= 20
            mssg = " CAMERA ELEVATION DOWN!"
            print(mssg)

        if key == GLUT_KEY_LEFT:
            curr_angl = math.atan2(y, x)
            step_angl = math.radians(10)
            new_angl = curr_angl - step_angl
            x = rad * math.cos(new_angl)
            y = rad * math.sin(new_angl)
            mssg = " CAMERA ORBITING LEFT!"
            print(mssg)

        if key == GLUT_KEY_RIGHT:
            curr_angl = math.atan2(y, x)
            step_angl = math.radians(10)
            new_angl = curr_angl + step_angl
            x = rad * math.cos(new_angl)
            y = rad * math.sin(new_angl)
            mssg = " CAMERA ORBITING RIGHT!"
            print(mssg)

        cam_xyz = [x, y, z]

def mouse_input(btn, state, x, y):
    """mouse input"""
    global fps_mode


    if btn == GLUT_LEFT_BUTTON and state == GLUT_DOWN:
        if not gme_end and not hax_on:
            shoot_goli()

    if btn == GLUT_RIGHT_BUTTON and state == GLUT_DOWN:
        fps_mode = not fps_mode
        if fps_mode:
            mssg = " SWITCHED TO FIRST PERSON VIEW!"
            print(mssg)
        else:
            mssg = " SWITCHED TO THIRD PERSON VIEW!"
            print(mssg)

def setup_cam():
    """camera setup"""
    global cam_xyz, fps_mode, auto_trgt, hax_on, hax_spin

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(fov_deg, scrn_w/scrn_h, 0.1, 1500)
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()

    if fps_mode:
        cam_shirt_hght = 60

        if hax_on and auto_trgt:
            curr_angl = hax_spin
        else:
            curr_angl = plyr_rot

        eye_x = plyr_x - 5 * math.sin(math.radians(curr_angl))
        eye_y = plyr_z - 5 * math.cos(math.radians(curr_angl))
        eye_z = cam_shirt_hght

        cntr_x = plyr_x + 50 * math.sin(math.radians(curr_angl))
        cntr_y = plyr_z + 50 * math.cos(math.radians(curr_angl))
        cntr_z = cam_shirt_hght

        gluLookAt(eye_x, eye_y, eye_z,
                  cntr_x, cntr_y, cntr_z,
                  0, 0, 1)
    else:
        x, y, z = cam_xyz
        gluLookAt(x, y, z,
                  0, 0, 0,
                  0, 0, 1)

def idle_func():
    """idle function"""
    updt_gme()
    glutPostRedisplay()

def display_scrn():
    """main display"""
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glLoadIdentity()
    glViewport(0, 0, scrn_w, scrn_h)

    setup_cam()

    # 3d elements
    render_floor_n_walls()
    render_baddies()
    render_shots()
    render_plyr()

    # ui elements
    draw_hud_info()
    draw_ctrl_info()

    if gme_end:
        draw_gme_end_box()
        draw_txt_at(40, 51, "MISSION FAILED", z=0.2)
        draw_txt_at(40, 48, "Press R to Retry", z=0.2)

    if hax_on:
        draw_txt_at(2, 80, " HACK MODE ACTIVE", z=0.1)

    if fps_mode:
        draw_txt_at(2, 75, " First Person", z=0.1)
    else:
        draw_txt_at(2, 75, " Third Person", z=0.1)

    glutSwapBuffers()

def main_func():
    """main function"""
    glutInit()
    glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB | GLUT_DEPTH)
    glutInitWindowSize(scrn_w, scrn_h)
    glutInitWindowPosition(0, 0)
    wnd = glutCreateWindow(b"Combat Arena 3D")

    make_enemy()

    glutDisplayFunc(display_scrn)
    glutKeyboardFunc(kybrd_input)
    glutSpecialFunc(spcl_key_input)
    glutMouseFunc(mouse_input)
    glutIdleFunc(idle_func)

    glutMainLoop()

if __name__ == "__main__":
    main_func()